In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import load_csv_data, predict_labels, create_csv_submission
from preprocessing import standardize_train, standardize_test, add_bias
import math

## Load data

In [67]:
y_train, input_train, ids_train = load_csv_data('train.csv', sub_sample=False)
print(y_train.shape)
print(input_train.shape)

(250000,)
(250000, 30)


In [68]:
y_test, input_test, ids_test = load_csv_data('test.csv', sub_sample=False)
print(y_test.shape)
print(input_test.shape)

(568238,)
(568238, 30)


## Pre-processing

In [104]:
#Remove -999 values

In [109]:
# IDEE: on pourrait ausi tenter build_polynomial comme dans les séries si on est motivés

In [110]:
x_train_no_999, y_train_no_999 = remove_999(input_train, y_train)

x_train_no_999, mean, std = standardize_train(x_train_no_999)
x_train_no_999 = add_bias(x_train_no_999)

x_test = standardize_test(input_test, mean, std)
x_test = add_bias(x_test)   #

In [111]:
print(x_train.shape)
print(x_test.shape)

(68114, 31)
(568238, 31)


## Should be useful

In [72]:
#besoin pour SGD, prise de lab02
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]      

In [73]:
# Celle du lab03, juste sans les degrés
def plot_train_test(train_errors, test_errors, lambdas):
    plt.semilogx(lambdas, train_errors, color='b', marker='*', label="Train error")
    plt.semilogx(lambdas, test_errors, color='r', marker='*', label="Test error")
    plt.xlabel("lambda")
    #plt.ylabel("RMSE")
    plt.ylabel("Loss")
    plt.title("Ridge regression for polynomial degree ")
    leg = plt.legend(loc=1, shadow=True)
    leg.draw_frame(False)
    #plt.savefig("ridge_regression")

# Methods to implement 
P.2 of project description

-  least_square
-  least_square_GD
-  least_square_SGD

-  ridge_regression
-  logistic_regression
-  reg_logistic_regression

All function should return (w, loss) <br>
Later add them in implementations.py

In [287]:
# Define the parameters of the algorithm. (probably change afterwards)
max_iters = 200                                          
gamma = 0.1
batch_size = 1

# Initialization
w_initial = np.random.rand(x_train.shape[1])

## Least square 

In [288]:
def compute_loss_ls(y, tx, w):
    """Calculate the loss.
    """
    e = y - tx.dot(w)
    #print(e)
    return (1/2)*np.mean(e**2)

In [289]:
e = compute_loss_ls(y_train, x_train, w_initial)
#print(e)

In [290]:
def least_squares(y, tx):
    """Compute the optimal w and the loss with least square technique"""
    """Required by project description"""
    a = tx.T.dot(tx) 
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)                #p.7 du cours least squares
    loss = compute_loss_ls(y, tx, w)                 #p.3 du cours least squares
    return w, loss

In [291]:
w_ls, loss_ls = least_squares(y_train, x_train)
print("w is {} \n of shape {}".format(w_ls, w_ls.shape))
print("\n loss of least squares is {}".format(loss_ls))

w is [-3.14664000e-01  2.93788272e-02 -2.52531475e-01 -2.54791124e-01
 -3.03696824e-02 -1.40144726e+00  2.95701641e-01 -1.07889472e+01
  2.67880862e-01 -2.44934971e-03 -3.28827529e+02 -1.82647888e-01
  1.14039627e-01  2.05045961e+01  6.38853340e+01 -3.18961885e-04
 -1.80884297e-03  6.29944860e+01 -4.48641524e-04  1.54379296e-03
  1.21462701e-01  3.95268757e-04 -6.33223475e-02 -2.06747093e-01
 -1.16655767e-01  9.86256328e-02  1.67907718e-01 -3.35146253e-02
 -2.98358685e+00 -5.36388099e+00  2.78478918e+02] 
 of shape (31,)

 loss of least squares is 0.33944559848933104


In [292]:
def compute_gradient_least_squares(y, tx, w):
    """Compute the gradient of least square."""
    '''
    print("y is {}".format(y.shape))
    print("tx is {}".format(tx.shape))
    print("w is {}".format(w.shape))
    a = tx.dot(w)
    print("a is {}".format(a.shape))
    '''
    err = y - w.dot(tx.T) #tx.dot(w)
    #print(err)
    #print(err.shape)
    grad = -tx.T.dot(err)/len(err)         #p.5 du cours least squares
    return grad                  

In [293]:
#compute_gradient_least_square(y_train, x_train, w)
grad_ls = compute_gradient_least_squares(y_train, x_train, w_initial)
print("grad is {} \n of shape {}".format(grad_ls, grad_ls.shape))
#print("\nerr of least squares is {}".format(err))

grad is [ 0.57961034  1.77414577 -0.33939429  0.91283755  5.05389876  6.09099835
  5.83435243  6.09203034 -1.91261717  3.19426975  5.85219796  1.22995334
  3.5394617   6.0914679   1.77543271  0.51719589  0.21840713  2.58515579
  0.70416239  0.85919932  3.50541424  0.78169237  5.56387894  6.42859867
  5.17323323  4.95835178  4.95687385  6.12262134  6.09169066  6.09123954
  5.92050984] 
 of shape (31,)


In [294]:
gamma = 0.1
def least_squares_GD(y, tx, w_initial, max_iters, gamma):
    """Gradient descent algorithm with least squares."""
    """Required by project description"""
    w = w_initial
    for n_iter in range(max_iters):
        # compute gradient and error
        grad = compute_gradient_least_squares(y, tx, w)
        #print(grad)
        # gradient w by descent update
        w = w - (gamma * grad)
        # print(w)
        # calculate loss    
    loss = compute_loss_ls(y, tx, w)           
    return w, loss

In [295]:
w_ls_gd, loss_ls_gd = least_squares_GD(y_train, x_train, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_ls_gd, w_ls_gd.shape))
print("\n loss of least squares is {}".format(loss_ls_gd))

w is [-0.314664    0.02358441 -0.23819999 -0.25217892  0.02891371 -0.30349336
  0.44807777  0.24192151  0.27693041 -0.02272495 -0.0035236  -0.16347312
  0.11111952  0.15236211  0.19513764 -0.00065166 -0.00112557  0.25392454
 -0.00077372  0.00238736  0.09579818  0.0008514  -0.04141987  0.10213103
  0.50571422 -0.13920452 -0.32519927 -0.31898239  0.03988202 -0.15732273
 -0.22661771] 
 of shape (31,)

 loss of least squares is 0.3420666358033106


In [296]:
gamma = 0.01
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """Stochastic Gradient Descent algorithm with least squares."""
    """Required by project description"""
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad = compute_gradient_least_squares(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
    loss = compute_loss_ls(y, tx, w)                                 #TO CHECK p.3 least squares
    return w, loss

In [297]:
w_ls_sgd, loss_ls_sgd = least_squares_SGD(y_train, x_train, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_ls_sgd, w_ls_sgd.shape))
print("\n loss of least squares is {}".format(loss_ls_sgd))

w is [-0.28032291  0.16570215  0.07666079 -0.25266048  0.04004951 -0.31582698
  0.40969289  0.23024454  0.399266    0.21669759 -0.14169261 -0.06445322
  0.20137663  0.13997472  0.28989413 -0.09077684  0.17035135  0.00634319
  0.14447043  0.03204641 -0.06508943  0.06896809  0.03411254  0.25646954
  0.44379835 -0.18862806 -0.37422311 -0.3594323   0.02784728 -0.16862433
 -0.35565734] 
 of shape (31,)

 loss of least squares is 0.5125444943059494


## Ridge regression

In [298]:
# Should do grid search  TO DO
lambda_ = 0.01

In [299]:
def compute_loss_ridge(y, tx, w, lambda_):
    """Not required specifically""" 
    err = y - tx.dot(w)
    loss = (1/2) * np.mean(err**2) + lambda_ * (np.linalg.norm(w,2))**2   #TO CHECK p.3 ridge regression
    return loss

In [300]:
def compute_gradient_ridge(y, tx, w, lambda_):
    """Not required specifically""" 
    err = y - tx.dot(w)
    grad = -tx.T.dot(err) / len(err) + (2 * lambda_* w)
    return grad, err

In [301]:
##################
# Je suis un peu confuse. Est ce que ce qu'il veut c'est ça ou bien ce que j'ai fait aux fonctions ridge_GD et ridge_SGD ?? 
# ridge_GD et ridge_SGD sont juste en-dessous
def ridge_regression(y, tx, lambda_):
    """Ridge regression."""
    """Required by project description"""
    lambd = 2 * tx.shape[0] * lambda_
    a = tx.T.dot(tx) + lambd * np.identity(tx.shape[1])
    b = tx.T.dot(y) 
    w = np.linalg.solve(a, b) 
    loss = compute_loss_ridge(y, tx, w, lambda_)   
    return w, loss

In [302]:
w_rreg, loss_rreg = ridge_regression(y_train, x_train, lambda_)
#print("w is {} \n of shape {}".format(w_rreg, w_rreg.shape))
#print("\n loss of ridge regression is {}".format(loss_rreg))

In [303]:
gamma = 0.1
def ridge_GD(y, tx, initial_w, max_iters, gamma, lambda_):
    """Gradient descent algorithm with Ridge."""
    """Not required specifically""" 
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and error
        grad, _ = compute_gradient_ridge(y, tx, w, lambda_)
        #print("grad is {} \n of shape {}".format(grad, grad.shape))
        #print("\n err of least squares is {}".format(err))
        # gradient w by descent update
        w = w - gamma * grad
        # calculate loss
    loss = compute_loss_ridge(y, tx, initial_w, lambda_)            
    return w, loss

In [304]:
w_r_gd, loss_r_gd = ridge_GD(y_train, x_train, w_initial, max_iters, gamma, lambda_)
#print("w is {} \n of shape {}".format(w_r_gd, w_r_gd.shape))
#print("\n loss of ridge_GD is {}".format(loss_r_gd))

In [305]:
gamma = 0.01
def ridge_SGD(y, tx, initial_w, max_iters, gamma, lambda_):
    """Stochastic Gradient Descent algorithm with least squares."""
    """Not required specifically""" #Implicitely requested I think
    
    w = initial_w
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = compute_gradient_ridge(y_batch, tx_batch, w, lambda_)    
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
    loss = compute_loss_ridge(y, tx, initial_w, lambda_) 
    return w, loss

In [306]:
w_r_sgd, loss_r_sgd = ridge_SGD(y_train, x_train, w_initial, max_iters, gamma, lambda_)
#print("w is {} \n of shape {}".format(w_r_sgd, w_r_sgd.shape))
#print("\n loss of ridge SGD is {}".format(loss_r_sgd))

## Logistic regression
Debugged but not sure from here: Il y aura probablement une correction de labo pour améliorer / re-vérifier les fonctions de logistic regression

### Il y a un NaN dans la loss à cause d'un outlier. Je sais pas si on choisis d'enlever ou comment traiter. On verra.

In [228]:
def sigmoid(tx, w):
    """Compute sigmoid function"""
    z = np.array(np.exp(-tx.dot(w)))
    return 1./(1 + z)

value = sigmoid(x_train, w_initial)
print(value)

[0.99981039 0.90820056 0.92786083 ... 0.28142199 0.00709906 0.04907172]


In [226]:
def compute_logreg_loss(y, tx, w):  #np.log parce que math.log fonctionne pas.. J'ai toujours pas compris pourquoi.
    """Compute error and gradient of logistic regression"""
    sig = sigmoid(tx, w)
    loss = np.sum((-y * np.log(sig) - (1-y) * np.log(1-sig)), axis = -1)/len(y)
    print(max(sig))
    #a = (-y * np.log(sig) - (1-y) * np.log(1-sig))
    #print(max(a))
    return loss

In [227]:
loss = compute_logreg_loss(y_train, x_train, w_initial)
print("loss is {}".format(loss))

[0.51459227 0.10073464 0.53488351 0.42669814 0.02003378 0.140064
 0.2264286  0.072293   0.49416143 0.6004996  0.14155126 0.00350222
 0.81835964 0.2093738  0.7436304  0.88667924 0.88977018 0.11193385
 0.70287969 0.70028424 0.34716633 0.66417114 0.94782936 0.73171242
 0.5700489  0.53273724 0.82729083 0.43374625 0.09599914 0.22535177
 0.38780099]
1.0
loss is nan


C:\Users\Pauli\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
C:\Users\Pauli\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.


In [119]:
def compute_logreg_grad(y, tx, w):
    """Compute error and gradient of logistic regression""" 
    sig = sigmoid(tx, w)
    err  = sig - y
    grad = tx.T.dot(err)/len(y)
    return grad

In [ ]:
grad_lr = compute_logreg_grad(y_train, x_train, w_initial)
print("grad is {} \n of shape {}".format(grad_lr, grad_lr.shape))

In [ ]:
gamma = 0.01
def logistic_regression(y, tx, initial_w, max_iters, gamma): #SGD  (GD easy to implement from here)
    """Stochastic Gradient Descent algorithm with logistic regression."""
    """Required by project description"""
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient
            grad = compute_logreg_grad(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # compute a stochastic loss
            compute_logreg_loss(y_batch, tx_batch, w)
    return w, loss

In [ ]:
w_lr, loss_lr = logistic_regression(y_train, x_train, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_lr, w_lr.shape))
print("\n loss of ridge_SGD is {}".format(loss_lr))

## Regularized Logistic Regression

In [ ]:
def compute_logreg_reg_loss(y, tx, w, lambda_):
    """Compute error and gradient of logistic regression"""
    reg = ( lambda_/(2*len(y)) ) * sum(w**2)
    loss = compute_logreg_loss(y, tx, w) + reg
    return loss

In [ ]:
loss_lrr = compute_logreg_reg_loss(y_train, x_train, w_initial, lambda_)
print("loss is {}".format(loss_lrr))

In [ ]:
def compute_logreg_reg_grad(y, tx, w, lambda_):
    """Compute error and gradient of logistic regression"""     
    grad = compute_logreg_grad(y, tx, w) 
    reg = (lambda_/len(y)) * w[1:]
    grad[1:] = grad[1:] + reg            
    return grad

In [ ]:
grad_lrr = compute_logreg_reg_grad(y_train, x_train, w_initial, lambda_)
print("grad is {} \n of shape {}".format(grad_lrr, grad_lrr.shape))

In [ ]:
###########
# La loss de ridge SGD me semble particulièrement haute. A vérifier avec les plots (et/ou code).
gamma = 0.01
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    """Stochastic Gradient Descent algorithm with REGULARIZED logistic regression."""
    """Required by project description"""
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient
            grad = compute_logreg_reg_grad(y_batch, tx_batch, w, lambda_)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # compute a stochastic loss
            loss = compute_logreg_reg_loss(y_batch, tx_batch, w, lambda_)
    return w_rlr_sgd, loss_rlr_sgd

In [ ]:
w_rlr_sgd, loss_rlr_sgd = reg_logistic_regression(y_train, x_train, lambda_, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_rlr_sgd, w_rlr_sgd.shape))
print("\n loss of ridge_SGD is {}".format(loss_rlr_sgd))

## Choisir méthode:
(D'après moi)
- Pour chaque méthode: faire un plot de test_error (=loss) pour un grand nombre d'iteration (assez pour que on voit que l'erreur commence à remonter si possible).  Il faudrait idealement voir la courbe descendre et monter dans le graphe (underfit à overfit)
- Garder la valeur de n iter ou la test error était minimale (au cas où: risque d'être très long pour ridge si lambda_ trop grand)
- Comparer les loss de chaque méthode à leur n_iter optimale
- La meilleure est celle ou c'est le plus faible

En plus comme ça on aura des beau plots et des arguments pour le rapport.

In [ ]:
#y_pred = predict_labels(weights, data)